In [ ]:
import cascade as csc
import numpy as np
import pykep as pk
from copy import deepcopy
from tqdm.notebook import tqdm
import heyoka as hy

In [ ]:
nparts = 17378

In [ ]:
# Dynamical variables.
x,y,z,vx,vy,vz = hy.make_vars("x","y","z","vx","vy","vz")

# Constants.
GMe = 3.986004407799724e+5
C20 = -4.84165371736e-4
Re = 6378.1363

# Create Keplerian dynamics.
dyn = csc.dynamics.kepler(mu = GMe)

# Add the J2 terms.
magR2 = hy.sum_sq([x, y, z])
J2term1 = GMe*(Re**2)*np.sqrt(5)*C20/(2*magR2**(1./2))
J2term2 = 3/(magR2**2)
J2term3 = 15*(z**2)/(magR2**3)
fJ2X = J2term1*x*(J2term2 - J2term3)
fJ2Y = J2term1*y*(J2term2 - J2term3)
fJ2Z = J2term1*z*(3*J2term2 - J2term3)
dyn[3] = (dyn[3][0], dyn[3][1] + fJ2X)
dyn[4] = (dyn[4][0], dyn[4][1] + fJ2Y)
dyn[5] = (dyn[5][0], dyn[5][1] + fJ2Z)

In [ ]:
dyn

In [ ]:
rng = np.random.default_rng()

x = []
y = []
z = []

vx = []
vy = []
vz = []

sizes = []

for _ in range(nparts):
    a = rng.uniform(1.02*Re, 1.3*Re)
    e = rng.uniform(0, 0.02)
    inc = rng.uniform(0, 0.05)
    om = rng.uniform(0, 2*np.pi)
    Om = rng.uniform(0, 2*np.pi)
    nu = rng.uniform(0, 2*np.pi)
    size = rng.uniform(1e-4, 1e-3)
    
    r, v = pk.par2ic([a, e, inc, om, Om, nu], GMe)
    
    x.append(r[0])
    y.append(r[1])
    z.append(r[2])

    vx.append(v[0])
    vy.append(v[1])
    vz.append(v[2])
    
    sizes.append(size)

sim = csc.sim(x,y,z,vx,vy,vz,sizes,0.23 * 806.81,dyn=dyn)

In [ ]:
final_t = 86400*365

In [ ]:
pbar = tqdm(total=final_t)

while sim.time < final_t:
    orig_time = sim.time
    
    oc = sim.step()
    
    pbar.update(sim.time - orig_time)
   
    if oc == csc.outcome.collision:
        # TODO different code needed for crash
        # on Earth here.
        pi, pj = sim.interrupt_info
        
        print("Collision detected, re-initing particles {} and {}".format(pi, pj))
        
        new_x = deepcopy(sim.x)
        new_y = deepcopy(sim.y)
        new_z = deepcopy(sim.z)
        
        new_vx = deepcopy(sim.vx)
        new_vy = deepcopy(sim.vy)
        new_vz = deepcopy(sim.vz)
        
        new_sizes = deepcopy(sim.sizes)

        for idx in [pi, pj]:
            a = rng.uniform(1.02*Re, 1.3*Re)
            e = rng.uniform(0, 0.02)
            inc = rng.uniform(0, 0.05)
            om = rng.uniform(0, 2*np.pi)
            Om = rng.uniform(0, 2*np.pi)
            nu = rng.uniform(0, 2*np.pi)
            size = rng.uniform(1e-4, 1e-3)

            r, v = pk.par2ic([a, e, inc, om, Om, nu], GMe)

            new_x[idx] = r[0]
            new_y[idx] = r[1]
            new_z[idx] = r[2]

            new_vx[idx] = v[0]
            new_vy[idx] = v[1]
            new_vz[idx] = v[2]
            
            new_sizes[idx] = size

        sim.set_new_state(new_x, new_y, new_z, new_vx, new_vy, new_vz, new_sizes)

pbar.close()
del pbar